# Taproot TapTree

- The following material adapted from the Bitcoin Optech [Schnorr Taproot workshop](https://bitcoinops.org/en/schorr-taproot-workshop/).

## Setup 
**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *
from functions.bip_0340_reference import *

## Introduction

Taproot transactions build off the previous transaction types (legacy and segwit v0), so it is helpful to be familiar with those already. With P2SH and P2WSH, you commit to a single script, that script must be revealed when the transaction is spent. With taproot, you only ever commit to a public key, which may or may not commit to a tapscript. (Note that even if the public key is not intended to be used with a tapscript, it's best to commit to an unspendable script.)

In the section on TapTweaks we covered how public keys can encode commitments by being tweaked. Now we will cover the structure of what get committed to the public key.

Unlike script hash transaction outputs, with TapTree you commit to the merkle root of a binary tree containing up to 128 levels. Each leaf encodes a script, similar to script hash outputs. A consequence of this is that rather than using an OP_IF condition in your script, it usually makes sense to contain the different spending conditions in different leafs of the tree. 

## Constructing a taptree

### Taptree binary tree commitments

Committing multiple tapscripts requires a commitment structure resembling merkle tree construction.

**The TapTree is different than the header merkle tree in the following ways:**

* Tapleaves can be located at different heights.
* Ordering of TapLeaves is determined lexicograpically.
* Location of nodes are tagged (No ambiguity of node type).
 
Internal nodes are called tapbranches, and are also computed with the `tagged_hash("Tag", input_data)` function.
 
Tagged hashes are particularly useful when building a taptree commitment. They prevent node height ambiguity currently found in the transaction merkle tree, which allows an attacker to create a node which can be reinterpreted as either a leaf or internal node. Tagged hashes ensure that a tapleaf cannot be misinterpreted as an internal node and vice versa.

In [2]:
# Tagged Hashes
# the 'tag' depends on where the tagged hash is being used e.g. 'TapTweak', 'TapBranch', TapScript'
def tagged_hash(tag, msg):
    tag_hash = sha256(tag.encode())
    return sha256(tag_hash + tag_hash + msg)

![test](../images/taproot-workshop/taptree0.jpg)

###  Compute a taptweak from a taptree

In the cell below, we will commit three pay-to-pubkey scripts to a taptweak and then derive the bech32m address. We will use the same merkle tree structure as in the previous illustration.

1. Compute TapLeaves A, B and C.
2. Compute Internal TapBranch AB.
3. Compute TapTweak
4. Derive the bech32m address.

In [3]:
TAPSCRIPT_VER = bytes([0xc0])  # This is currently the only tapscript version. In future there may be others.
internal_privkey = bytes.fromhex("83a5f1039118fbb4276cac2db41d236c1c1790d97d955c228fa3bde439fbec2a")
internal_pubkey = tr.pubkey_gen(internal_privkey)

# Derive three private/public (x-only) key pairs
privkeyA = bytes.fromhex("1059bf26660804ced9a3286a16497d7e70692d14dc04e1220c2dbef3667b74f7")
pubkeyA = tr.pubkey_gen(privkeyA)
privkeyB = bytes.fromhex("2b22bf11ab862a35f16301c0afc7afe60f66d31fc29645f79c2ab43655e65d33")
pubkeyB = tr.pubkey_gen(privkeyB)
privkeyC = bytes.fromhex("7f8b28e51da049bf63e31d3a3261579c0f5c1fc8058c65a79482814e5061f9f6")
pubkeyC = tr.pubkey_gen(privkeyC)

# Create corresponding pubkey scripts
scriptA = b"\x20" + pubkeyA + b"\xac"
scriptB = b"\x20" + pubkeyB + b"\xac"
scriptC = b"\x20" + pubkeyC + b"\xac"

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# 1) Compute TapLeaves A, B and C.
# Method: pushbytes(data) is a function which adds compactsize to input data.
hash_inputA =  TAPSCRIPT_VER + pushbytes(scriptA)
hash_inputB =  TAPSCRIPT_VER + pushbytes(scriptB)
hash_inputC =  TAPSCRIPT_VER + pushbytes(scriptC)
taggedhash_leafA =  tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB =  tagged_hash("TapLeaf", hash_inputB)
taggedhash_leafC =  tagged_hash("TapLeaf", hash_inputC)

# 2) Compute Internal node TapBranch AB.
internal_nodeAB = tapbranch_hash(taggedhash_leafA, taggedhash_leafB)

# 3) Compute TapTweak.
rootABC =  tapbranch_hash(internal_nodeAB, taggedhash_leafC)
taptweak =  tagged_hash("TapTweak", internal_pubkey + rootABC)

# 4) Derive the bech32m address.
negated, taproot_pubkey = taproot_tweak_pubkey(internal_pubkey, rootABC)
print("TapTweak:", taptweak.hex())
spk = bytes.fromhex("5120") + taproot_pubkey
bech32m_address = spk_to_bech32(spk, 'regtest')
print('Bech32m address:', bech32m_address)

TapTweak: 8e53fc0da2e9e8e27404703010ae519e85576ef6c73dde1b2bffac7f17b114a9
Bech32m address: bcrt1p5jhcyymfj7tkgv0jca4pz7tx9uzvznu0mlfymey6ph63f9h8x0gs7683vc


## Quiz


 ## Answers
    

## Exercise
